# Text to SQL with smolagents

[https://huggingface.co/learn/cookbook/agent_text_to_sql](https://huggingface.co/learn/cookbook/agent_text_to_sql)

# Doc is missing imports O_o

In [2]:
!pip install smolagents datasets langchain sentence-transformers langchain-community --upgrade -q

In [3]:
!pip install sqlalchemy

A standard text-to-sql pipeline is brittle, since the generated SQL query can be incorrect. Even worse, the query could be incorrect, but not raise an error, instead giving some incorrect/useless outputs without raising an alarm.

👉 Instead, an agent system is able to critically inspect outputs and decide if the query needs to be changed or not, thus giving it a huge performance boost.

Let’s build this agent! 💪

Setup SQL tables

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create city SQL table
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

In [5]:
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

Let’s check that our system works with a basic query:



In [7]:
with engine.connect() as con:
    rows = con.execute(text("""SELECT * from receipts"""))
    for row in rows:
        print(row)

(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)


# Build our agent

Now let’s make our SQL table retrievable by a tool.

Our sql_engine tool needs the following: (read the documentation for more detail)

- A docstring with an Args: part. This docstring will be parsed to become the tool’s description attribute, which will be used as the instruction manual for the LLM powering the agent, so it’s important to provide it!
- Type hints for inputs and output.

In [6]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

# DOC FORGETS TO LOGIN FOR API ACCESS O_o

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("meta-llama/Meta-Llama-3-8B-Instruct"),
)

In [10]:
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Meta-Llama-3-8B-Instruct ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
                                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: None

[Step 0: Duration 0.27 seconds| Input tokens: 2,086 | Output tokens: 1]

...

[Step 5: Duration 0.10 seconds| Input tokens: 13,507 | Output tokens: 8]

Reached max steps.

Final answer: I apologize for the previous errors. Based on the available information, I will try to answer the 
original question.

To find the client with the most expensive receipt, I will need to access the database or a data storage system to 
retrieve the necessary information.

Assuming we have a database or a data storage system with the required information, I can query it to find the 
client with the most expensive receipt.

Here's a possible query:

```
SELECT client_name, SUM(receipt_amount) as total_spent
FROM receipts
GROUP BY client_name
ORDER BY total_spent DESC
LIMIT 1;
```

This query will return the name of the client who has spent the most, along with the total amount they have spent.

Please note that this is a hypothetical query, and the actual implementation may vary depending on the specific 
database or data storage system being used.

If you have any further information or clarification regarding the database or data storage system, I'll be happy 
to help.

[Step 6: Duration 0.00 seconds| Input tokens: 13,968 | Output tokens: 206]

"I apologize for the previous errors. Based on the available information, I will try to answer the original question.\n\nTo find the client with the most expensive receipt, I will need to access the database or a data storage system to retrieve the necessary information.\n\nAssuming we have a database or a data storage system with the required information, I can query it to find the client with the most expensive receipt.\n\nHere's a possible query:\n\n```\nSELECT client_name, SUM(receipt_amount) as total_spent\nFROM receipts\nGROUP BY client_name\nORDER BY total_spent DESC\nLIMIT 1;\n```\n\nThis query will return the name of the client who has spent the most, along with the total amount they have spent.\n\nPlease note that this is a hypothetical query, and the actual implementation may vary depending on the specific database or data storage system being used.\n\nIf you have any further information or clarification regarding the database or data storage system, I'll be happy to help."

# Retry with more powerful model

Qwen O_o

In [11]:
agent2 = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("Qwen/QwQ-32B-Preview"),
)

In [12]:
agent2.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ HfApiModel - Qwen/QwQ-32B-Preview ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
500 Server Error: Internal Server Error for url: 
https://api-inference.huggingface.co/models/Qwen/QwQ-32B-Preview/v1/chat/completions (Request ID: 
I0bJOjkWgbAExRXiNKW06)

Model too busy, unable to get response in less than 60 second(s)

[Step 0: Duration 60.04 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts WHERE price + tip = (SELECT MAX(price + tip) FROM receipts);"        
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out - Final answer: 
('Woodrow Wilson',)

[Step 2: Duration 50.95 seconds| Input tokens: 2,128 | Output tokens: 1,150]

"\n('Woodrow Wilson',)"

# Not sure what is failing here, doesnt convert to code it seems

- update, rerun notebook and now the qwen 32b seems to work ok, before it was just generating plain text in english

# HERE IS HOW TO FIND THE SO CALLED warm MODELS FROM HfApiModel LIST

docs are so unclear

[https://huggingface.co/models?inference=warm&pipeline_tag=text-generation](https://huggingface.co/models?inference=warm&pipeline_tag=text-generation)

**NOTE: that the Llama stuff requires login and auth, so im using the QWEN ones, also the 72B is free it seems**

In [13]:
agent3 = CodeAgent(
    tools=[sql_engine],
    model=HfApiModel("Qwen/Qwen2.5-72B-Instruct"),
)

In [14]:
agent3.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT customer_name FROM receipts ORDER BY price DESC LIMIT 1"                                         
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

('Woodrow Wilson',)

Out: None

[Step 0: Duration 4.96 seconds| Input tokens: 2,128 | Output tokens: 70]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Woodrow Wilson")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Woodrow Wilson

[Step 1: Duration 3.69 seconds| Input tokens: 4,422 | Output tokens: 119]

'Woodrow Wilson'

# Try a more complex example

- i wanted to do this with the 72B Qwen but now i get too many requests error O_o

In [15]:
agent3.run("What is the total sum of the tips from all the customers whose first name begins with the letter A ?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the total sum of the tips from all the customers whose first name begins with the letter A ?            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  query = "SELECT SUM(tip) FROM receipts WHERE customer_name LIKE 'A%'"                                            
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

(1.44,)

Out: None

[Step 0: Duration 4.07 seconds| Input tokens: 2,133 | Output tokens: 79]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  total_sum = result[0]                                                                                            
  final_answer(total_sum)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 

[Step 1: Duration 2.97 seconds| Input tokens: 4,445 | Output tokens: 139]

'\n'

## Reasoning seems OK but format at the end/doesn't access the numerical value 1.44 that it found OK before

### Continue the cookbook example

Increasing difficulty: Table joins
Now let’s make it more challenging! We want our agent to handle joins across multiple tables.

So let’s make a second table recording the names of waiters for each receipt_id!

In [16]:
table_name = "waiters"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

We need to update the SQLExecutorTool with this table’s description to let the LLM properly leverage information from this table.

In [17]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


## COULDNT UNDERSTAND WHY IN EXAMPLE HE DOESN'T NEED TO RESPECIFY THE Args: PART

it seems it must parse up to Args into a `description` key, then keep args separate? need to read source

In [18]:
sql_engine.description # should still be the old description

"Allows you to perform SQL queries on the table. Returns a string representation of the result.\nThe table is named 'receipts'. Its description is as follows:\n    Columns:\n    - receipt_id: INTEGER\n    - customer_name: VARCHAR(16)\n    - price: FLOAT\n    - tip: FLOAT"

In [21]:
sql_engine.description = updated_description # <---- here updates description

print(sql_engine.description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


In [22]:
complex_agent = CodeAgent(
    tools=[sql_engine], # <--- this is updated now with descriptions of the 2 tables
    model=HfApiModel("Qwen/Qwen2.5-72B-Instruct"),
)

complex_agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT w.waiterName, SUM(r.tip) as total_tips                                                                    
  FROM receipts r                                                                                                  
  JOIN waiters w                                                                                                   
  ON r.receipt_id = w.receipt_id                                                                                   
  GROUP BY w.waiterName;                                                                                           
  """                                                                                                              
  result = sql_engine(query=query)                                                                                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

(sqlite3.OperationalError) no such column: w.waiterName
[SQL: 
SELECT w.waiterName, SUM(r.tip) as total_tips
FROM receipts r
JOIN waiters w
ON r.receipt_id = w.receipt_id
GROUP BY w.waiterName;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

[Step 0: Duration 5.45 seconds| Input tokens: 2,150 | Output tokens: 115]

...

[Step 5: Duration 6.71 seconds| Input tokens: 17,400 | Output tokens: 621]

Reached max steps.

Final answer: The waiter who got the most total money from tips is Michael Watts with $5.67.

[Step 6: Duration 0.00 seconds| Input tokens: 19,252 | Output tokens: 642]

'The waiter who got the most total money from tips is Michael Watts with $5.67.'

# Checking answer

```
(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)
```

```
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
```

So Corey is 1.2

Michael is 0.24+5.43=5.67

Margaret (he uses name for waiter and client O_o) is 1.0

**So gold answer is michael watts 5.67**